In [23]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt

In [43]:
# Reading in data from CSV
df = pd.read_csv("data/cfb_attendance.csv", encoding="cp1252")

# Change column datatypes
df['Date'] = df['Date'].astype('datetime64')
df['Team'] = df['Team'].astype('string')
df['Rank'] = df['Rank'].astype('string')
df['Time'] = df['Time'].astype('datetime64')
df['Opponent'] = df['Opponent'].astype('string')
df['Site'] = df['Site'].astype('string')
df['TV'] = df['TV'].astype('string')
df['Result'] = df['Result'].astype('string')

#df.dtypes

print(df['Result'])

0        W 38–0
1       W 38–31
2       W 28–21
3        L 7–38
4        W 52–6
         ...   
6667    L 20–27
6668    L 24–27
6669     L 0–19
6670    L 32–48
6671     L 7–39
Name: Result, Length: 6672, dtype: string


In [47]:
# Get home, away, and total scores

# Split the Result column on the first space to get a list
# of the result at 0 and the score (Home-Away) at 1
df['Home Score'] = df['Result'].str.split(None, 1)

# Take the 1st position to get the score (Home-Away) at 1
df['Home Score'] = df['Home Score'].str[1]

# Split the score by hyphen
df['Home Score'] = df['Home Score'].str.split("–", 1)

# Away score is at the 0 position
df['Away Score'] = df['Home Score'].str[1]

# Home Score is at the 1 position
df['Home Score'] = df['Home Score'].str[0]

# Get the result of the game: W = Home team win, L = Home team looses
df['Result'] = df['Result'].str[0]


df[['Home Score', 'Away Score', 'Result']]

,Home Score,Away Score,Result
0,38,0,W
1,38,31,W
2,28,21,W
3,7,38,L
4,52,6,W
...,...,...,...
6667,20,27,L
6668,24,27,L
6669,0,19,L
6670,32,48,L
